## Import

In [1]:
import os
import cv2
import io
import glob
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow import keras
from tensorflow.keras import layers, models, Sequential, backend
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import Concatenate, Lambda, Input, ZeroPadding2D, AveragePooling2D
from keras.utils.np_utils import to_categorical

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import ipywidgets as widgets
from IPython.display import clear_output,display

config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))
sess = tf.compat.v1.Session(config=config)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


SEED=6
tf.random.set_seed(SEED)

2022-12-22 15:57:00.197381: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 15:57:01.239193: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22320 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:ce:00.0, compute capability: 8.6


In [4]:
import re

def is_in(full_str, sub_str):
    if re.findall(sub_str, full_str):
        return True
    else:
        return False

## Build The DenseNet121

In [5]:
# 引入SeNet,增强通道注意力
def senet_block(inputs, ratio):
    shape = inputs.shape
    channel_out = shape[-1]
    # print(shape)
    # (2, 28, 28, 32) , [1,28,28,1], [1,28,28,1]
    squeeze = layers.GlobalAveragePooling2D()(inputs)
    # [2, 1, 1, 32]
    # print(squeeze.shape)
    # 第二层，全连接层
    # [2,32]
    # print(squeeze.shape)
    shape_result = layers.Flatten()(squeeze)
    # print(shape_result.shape)
    # [32,2]
    shape_result = layers.Dense(int(channel_out / ratio), activation='relu')(shape_result)
    # shape_result = layers.BatchNormalization()(shape_result)
    # [2,32]
    shape_result = layers.Dense(channel_out, activation='sigmoid')(shape_result)
    # shape_result = layers.BatchNormalization()(shape_result)

    # 第四层，点乘
    # print('heres2')
    excitation_output = tf.reshape(shape_result, [-1, 1, 1, channel_out])
    # print(excitation_output.shape)
    h_output = excitation_output * inputs
    return h_output



# Conv_Block：卷积单元 BN-RELU-CONV(1X1) —— BN-RELU-CONV(3X3) —— SeNet
def conv_block(x, nb_filter, dropout_rate=None, name=None, r=16):
    
    inter_channel = nb_filter*4

    # 1x1 convolution， 
    # epsilon 将小浮点数添加到方差以避免被零除
    # data_format="channels_first"，(batch_size, channels,height, width)则在BatchNormalization中设置axis=1，默认是channels_last，则设置axis=3
    x = BatchNormalization(epsilon=1.1e-5, axis=3, name=name+'_bn1')(x)
    x = Activation('relu', name=name+'_relu1')(x)
    x = Conv2D(inter_channel, 1, 1, name=name+'_conv1', use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    # 3x3 convolution
    x = BatchNormalization(epsilon=1.1e-5, axis=3, name=name+'_bn2')(x)
    x = Activation('relu', name=name+'_relu2')(x)
    x = ZeroPadding2D((1, 1), name=name+'_zeropadding2')(x)
    x = Conv2D(nb_filter, 3, 1, name=name+'_conv2', use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)
    
    # 加SE注意力机制
    # x = Squeeze_excitation_layer(r)(x)
    x= senet_block(x, r)
    return x


# Dense_Block 卷积组：由N个卷积单元组构成Dense Block
def dense_block(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, 
                grow_nb_filters=True, name =None, r = 16):

    concat_feat = x # store the last layer output

    for i in range(nb_layers):
        
        branch = i+1
        x =conv_block(concat_feat, growth_rate, dropout_rate, name=name+str(stage)+'_block'+str(branch), r=r) # 在参考的基础，修改的地方这里应该是相同的growth_rate=32
        concat_feat = Concatenate(axis=3, name=name+str(stage)+'_block'+str(branch))([concat_feat, x])

        
        if grow_nb_filters:
            nb_filter += growth_rate

    return concat_feat, nb_filter



# Transition_Block： 每个Dense block之间的传递block，主要用来降低特征层数
def transition_block (x,stage, nb_filter, compression=1.0, dropout_rate=None, name=None):

    x = BatchNormalization(epsilon=1.1e-5, axis=3, name=name+str(stage)+'_bn')(x)
    x = Activation('relu', name=name+str(stage)+'_relu')(x)
    
    x = Conv2D(int(nb_filter*compression), 1, 1, name=name+str(stage)+'_conv', use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)
    
    x = AveragePooling2D((2,2), strides=(2,2), name=name+str(stage)+'_pooling2d')(x)

    x= senet_block(x, 16)
    return x



# Se-DenseNet: 121
def DenseNet(nb_dense_block=4, growth_rate=32, nb_filter=64, reduction=0.0, dropout_rate=0.0, weight_decay=1e-4, classes=1000, weights_path=None, r=16):

    compression = 1.0-reduction
    nb_filter=64
    nb_layers = [6, 12, 24, 16]  #For DenseNet-121
    
    img_input = Input(shape=(224,224,3))

    #initial convolution
    x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)        # (3, 3)  s=1
    x = Conv2D(nb_filter, 7, 2, name='conv1', use_bias=False)(x)          # ====(7, 7)  s=2==== Conv2D
    x = BatchNormalization(epsilon=1.1e-5, axis=3, name='conv1_bn')(x)    
    x = Activation('relu', name='relu1')(x)
    x = ZeroPadding2D((1, 1), name='pool1_zeropadding')(x)                # (1, 1)  s=1
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)             # ====(3, 3 ) s=2==== MaxPooling

    #Dense block and Transition layer
    for block_id in range(nb_dense_block-1):
        stage = block_id+2 # start from 2

        x, nb_filter = dense_block(x, stage, nb_layers[block_id], nb_filter, growth_rate, 
        dropout_rate=dropout_rate, name='Dense', r=r)

        x = transition_block(x, stage, nb_filter, compression=compression, dropout_rate=dropout_rate, name='Trans')
        nb_filter *=compression
    
    final_stage = stage + 1
    x, nb_filter=dense_block(x, final_stage, nb_layers[-1], nb_filter, growth_rate, 
        dropout_rate=dropout_rate, name='Dense')

    # top layer
    x = BatchNormalization(name= 'final_conv_bn')(x)
    x = Activation('relu', name='final_act')(x)
    x = GlobalAveragePooling2D(name='final_pooling')(x)
    # x = Dense(classes, activation='softmax', name='fc')(x)

    model=models.Model(img_input, x, name='DenseNet_SE')

    return model


conv_base = DenseNet(reduction=.5)
path_weights = "densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
conv_base.load_weights(path_weights, by_name=True, skip_mismatch=True)
conv_base.trainable = True

model=conv_base.output

model=tf.keras.layers.Dense(512,activation='relu')(model)
model=tf.keras.layers.Dropout(0.5)(model)
model=tf.keras.layers.Dense(2,activation="softmax")(model)
model=tf.keras.models.Model(inputs=conv_base.input,outputs=model)

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()

2022-12-22 15:57:18.039100: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22320 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:ce:00.0, compute capability: 8.6


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_zeropadding (ZeroPadding2 (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9408        conv1_zeropadding[0][0]          
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
______________________________________________________________________________________________

2022-12-22 15:57:22.680903: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-12-22 15:57:22.680965: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-12-22 15:57:22.681046: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 1 GPUs
2022-12-22 15:57:22.681625: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /root/miniconda3/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-12-22 15:57:22.954241: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-12-22 15:57:22.954719: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
